In [1]:
!pip install transformers datasets
!pip install requirements.txt
!pip install gradio
!pip install opencv-python
!pip install tifffile matplotlib
!pip install rasterio
!pip install torch --upgrade
import cv2
from rasterio.plot import show
import tifffile as tiff
from PIL import Image
import rasterio
import matplotlib.pyplot as plt
import glob

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbconvert 7.16.4 requires markupsafe>=2.0, but you have markupsafe 1.1.0 which is incompatible.
notebook 7.2.2 requires jupyterlab<4.3,>=4.2.0, but you have jupyterlab 4.3.0 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
ERROR: Could not find a version that satisfies the requirement requirements.txt (from versions: none)
HINT: You are attempting to install a package literally name

In [ ]:
image1 = Image.open('examples/ImageLeft/04_05_0011_l.png')
image2 = Image.open('examples/ImageRight/04_05_0011_r.png')
image1 = np.array(image1)
image2 = np.array(image2)


result1_path = 'output/result1.png'
image3 = Image.open('examples/ImageLeft/04_05_0011_l.png')
image4 = Image.open('examples/ImageRight/04_05_0011_r.png')
dis_first = Image.open('examples/04_05_0011.png')
image3 = image3.convert('L')
image4 = image4.convert('L')


dis_first = np.array(dis_first)
dis_first = dis_first.astype(float) / 100


image3 = np.array(image3)
image4 = np.array(image4)
image3 = cv2.medianBlur(image3,5)
image4 = cv2.medianBlur(image4,5)


plt.imshow(image3, cmap='plasma')
plt.colorbar()
plt.show()

plt.imshow(image4, cmap='plasma')
plt.colorbar()
plt.show()


disparity = get_disp(image3, image4, result1_path)
plt.imshow(dis_first, cmap='plasma')
plt.colorbar()
plt.show()


result1_path = 'output/result1.png'


# Open an image file
image5 = Image.open('examples/ImageLeft/Screenshot from 2024-10-09 09-46-04.png')
image6 = Image.open('examples/ImageRight/Screenshot from 2024-10-09 09-46-16.png')
# Define the crop area (left, upper, right, lower)
# Crop 100 pixels from the right side
crop_area = (100, 0, image5.width - 800, image5.height-101)

# Crop the image
image5 = image5.crop(crop_area)

# Show the cropped image

crop_area = (0, 0, image6.width-900, image6.height-100)

# Crop the image
image6 = image6.crop(crop_area)

# Show the cropped image
# Save the cropped image

dis_second =Image.open('examples/04_05_0009.png')
image5.save('cropped_image5.png')
image6.save('cropped_image6.png')

image5 = np.array(image5)
image6 = np.array(image6)

dis_second = np.array(dis_second)
dis_second = dis_second.astype(float) / 100

plt.imshow(image5, cmap='plasma')
plt.colorbar()
plt.show()

plt.imshow(image6, cmap='plasma')
plt.colorbar()
plt.show()


disparity = get_disp(image5, image6, result1_path)

plt.imshow(dis_second, cmap='plasma')
plt.colorbar()
plt.show()

In [ ]:
image_res = Image.open('examples/04_05_0011.png')
image_res = np.array(image_res)
image_res = image_res.astype(float) / 100

plt.axis('off')
plt.imshow(image_res, cmap='plasma')
plt.colorbar()
plt.show()


# Path to the folder containing the .tif files
tif_files = glob.glob('examples/01.tif')


def ras_show_flip(file_path):
    with rasterio.open(file_path) as src:

        # Read a small window from the file
        window = src.read(window=((0, 3000), (0, 3000)))

        # Check if the window is 2D (single band) or 3D (multiple bands)
        if window.ndim == 3:
            # First rotate the window by 90 degrees clockwise (for 3D array)
            rotated_window = np.rot90(window, k=3, axes=(1, 2))  # Rotate along height and width axes

            # Then flip the rotated window upside-down (axis 1)
            flipped_rotated_window = np.flip(rotated_window, axis=1)  # Flip vertically
            flipped_rotated_window = np.rot90(flipped_rotated_window, k=3, axes=(1, 2))  # Rotate along height and width axes

            # Display the corrected image from band 1
            plt.imshow(flipped_rotated_window[0])
            plt.title('Corrected Image - Band 1')
            plt.axis('off')
            plt.show()

            # Display the corrected image from band 3 (if available)
            if flipped_rotated_window.shape[0] > 2:  # Ensure there are enough bands
                plt.imshow(flipped_rotated_window[2])
                plt.title('Corrected Image - Band 3')
                plt.axis('off')
                plt.show()

        elif window.ndim == 2:
            # If the window is 2D (single band), rotate and flip accordingly
            rotated_window = np.rot90(window[0], k=3)  # Rotate 90 degrees clockwise
            flipped_rotated_window = np.flip(rotated_window, axis=1)  # Flip vertically
            rotated_window = np.rot90(window[0], k=3)  # Rotate 90 degrees clockwise

            # Display the corrected image
            plt.imshow(flipped_rotated_window)
            plt.title('Corrected Image')
            plt.axis('off')
            plt.show()

        # Print metadata about the file
        print(f"File Dimensions: {src.width} x {src.height}")
        print(f"Number of Bands: {src.count}")
        print(f"Data Type: {src.dtypes[0]}")
def load_stereo_images(file_path):
    with rasterio.open(file_path) as src:

        # Read the two bands (assuming band 1 is left and band 2 is right)
        left_image = src.read(1)  # Band 1 (left image)
        right_image = src.read(5)  # Band 2 (right image)

        # Display left and right images side by side
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        axes[0].imshow(left_image, cmap='gray')
        axes[0].set_title('Left Image')
        axes[0].axis('off')

        axes[1].imshow(right_image, cmap='gray')
        axes[1].set_title('Right Image')
        axes[1].axis('off')

        plt.show()

        return left_image, right_image
def ras_show(file_path):
    with rasterio.open(file_path) as src:

        # Read a small window to check the data
        window_left = src.read(window=((0, 8000), (0, 8000)))
        rgb_left = window_left[:3].transpose((1, 2, 0))  # Transpose to shape (height, width, 3)

        plt.imshow(rgb_left)
        plt.title('Sample Window of the .tif File')
        plt.show()
        window_right = src.read(window=((0, 3000), (0, 3000)))
        rgb_right = window_right[:3].transpose((1, 2, 0))  # Transpose to shape (height, width, 3)

        plt.imshow(rgb_right)
        plt.title('Sample Window of the .tif File')
        plt.show()
        return rgb_left, rgb_right
    
file_path_one = 'examples/10030060.tif'
file_path_two = 'examples/04.tif'
image3, image4 = ras_show(file_path_two)

In [ ]:
# Open a multi-page TIFF file
tif = Image.open("examples/04.tif")
# Loop through pages/images in the TIFF
for i in range(tif.n_frames):
    tif.seek(i)
    tif.save(f"page_{i}.png")  # Save each image as a separate file

In [ ]:
import rasterio
import matplotlib.pyplot as plt

def load_stereo_images_from_window(file_path, left_window, right_window):
    with rasterio.open(file_path) as src:
        # Read the left image from the specified window
        left_image = src.read(1)

        # Rotate the image by 360 degrees
        (h, w) = left_image.shape[:2]
        center = (w // 2, h // 2)

        # Compute the rotation matrix
        rotation_matrix = cv2.getRotationMatrix2D(center, 360, 1.0)

        # Apply the rotation
        left_image = cv2.warpAffine(left_image, rotation_matrix, (w, h))
        print(left_image.shape)
        # Read band 1 for left image
        #left_image = color_normalization(left_image)

        # Read the right image from the specified window
        right_image = src.read(1)
        print(right_image.shape)

        # Read band 1 for right image
        #right_image = color_normalization(right_image)

        # Display left and right images side by side
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        axes[0].imshow(left_image)
        axes[0].set_title('Left Image')
        axes[0].axis('off')

        axes[1].imshow(right_image)
        axes[1].set_title('Right Image')
        axes[1].axis('off')

        plt.show()
        return left_image, right_image
# Define your file path
file_path = 'examples/04.tif'

# Load and display stereo images
image1, image2=load_stereo_images_from_window(file_path, left_window, right_window)

# Save the image
image1 = image1.astype(np.uint8)  # Convert to uint8 format (required for images)
print(image1)

# Save the image using OpenCV
cv2.imwrite('image1.png', image1)


# Save the image
image2 = image2.astype(np.uint8)  # Convert to uint8 format (required for images)
print(image2)
# Save the image using OpenCV
cv2.imwrite('image2.png', image2)

result1_path = 'output/result1.png'

disparity = get_disp(image1, image2, result1_path)

In [ ]:
import numpy as np
import cv2
import concurrent.futures
from PIL import Image
model = load_model()

# Load the model only once
def process_stereo_video_for_disparity(video_left_path, video_right_path, output_video_path=None, skip_frames=4):
    # Step 1: Open the left and right video streams
    cap_left = cv2.VideoCapture(video_left_path)
    cap_right = cv2.VideoCapture(video_right_path)
    i = 0

    # Get video properties
    frame_width = int(cap_left.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap_left.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap_left.get(cv2.CAP_PROP_FPS) // skip_frames  # Adjust fps according to skipped frames

    print("Frame width:", frame_width)
    print("Frame height:", frame_height)
    print("FPS:", fps)

    # Video writer to save the output, if specified
    if output_video_path:
        fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Using XVID codec
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    def process_frame_pair(frame_left, frame_right):
        # Convert frames to PIL format (if needed by your stereo model)
        image_left = Image.fromarray(cv2.cvtColor(frame_left, cv2.COLOR_BGR2RGB))
        image_right = Image.fromarray(cv2.cvtColor(frame_right, cv2.COLOR_BGR2RGB))
        # Get disparity map for the current frame

        disparity_map = inference(image_left, image_right, model, task='stereo')

        disparity_map = np.array(disparity_map)

        # Normalize disparity map to the range 0-255
        disparity_visual = (disparity_map / np.max(disparity_map) * 255).astype(np.uint8)

        # Apply color map for better visualization
        disparity_color = cv2.applyColorMap(disparity_visual, cv2.COLORMAP_PLASMA)

        # Save the disparity frame to the video file
        if output_video_path:
            out.write(disparity_color)

        return disparity_color

    # Step 2: Process each frame (using multithreading)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        while cap_left.isOpened() and cap_right.isOpened():
            i += 1

            # Read frames from both the left and right video streams
            ret_left, frame_left = cap_left.read()
            ret_right, frame_right = cap_right.read()  # Now correctly unpacking

            # Check if both frames were successfully read
            if not ret_left or not ret_right:
                print("End of video or error in reading frames.")
                break

            # Skip frames for faster processing
            for _ in range(skip_frames - 1):
                cap_left.grab()
                cap_right.grab()

            # Submit the frame pair to the thread pool for processing
            future = executor.submit(process_frame_pair, frame_left, frame_right)

            # Retrieve processed disparity map from future and show it
            disparity_color = future.result()

            # Optionally: show the disparity map in real-time
            #cv2.imshow('Disparity Map', disparity_color)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Step 3: Release all resources
    cap_left.release()
    cap_right.release()
    print(f"Processed {i} frames.")

    if output_video_path:
        out.release()
    cv2.destroyAllWindows()

# Example usage
video_left_path = 'examples/filename1 (1).avi'
video_right_path = 'examples/filename1 (1).avi'
output_video_path = 'output/depth_video.avi' 

# Process the stereo video and save the disparity output
process_stereo_video_for_disparity(video_left_path, video_right_path, output_video_path=output_video_path, skip_frames=3)

In [ ]:
def read_pfm(file):
    with open(file, 'rb') as f:
        # Read header
        header = f.readline().decode('utf-8').rstrip()
        if header == 'PF':
            color = True
        elif header == 'Pf':
            color = False
        else:
            raise Exception('Not a PFM file.')

        # Read dimensions
        dims = f.readline().decode('utf-8').rstrip()
        width, height = map(int, dims.split())

        # Read scale factor (negative for little endian)
        scale = float(f.readline().decode('utf-8').rstrip())
        endian = '<' if scale < 0 else '>'

        # Read the data
        data = np.fromfile(f, endian + 'f')
        shape = (height, width, 3) if color else (height, width)
        data = np.reshape(data, shape)
        data = np.flipud(data)

    return data


pfm_data_one = read_pfm('examples/DispLeft/dis0010_L.pfm')
pfm_data_two = read_pfm('examples/DispLeft/dis0100_L.pfm')
pfm_data_three = read_pfm('examples/DispLeft/dis0040_L.pfm')
pfm_data_four = read_pfm('examples/DispLeft/dis0120_L.pfm')
depth_one_true = get_depth(350, 15000, 10.3,10.3, pfm_data_one)  #focal_length, baseline, cx2, cx1, disparity
depth_two_true= get_depth(350, 15000, 10.3,10.3, pfm_data_two)
depth_three_true = get_depth(350, 15000, 10.3,10.3, pfm_data_three)  #focal_length, baseline, cx2, cx1, disparity
depth_four_true= get_depth(350, 15000, 10.3,10.3, pfm_data_four)

In [ ]:
image3 = tiff.imread('examples/po_97258_pan_0000000.tif')
image4 = tiff.imread('examples/po_97258_pan_0010000.tif')
plt.imshow(image3,cmap='gray')
plt.show()

plt.imshow(image4, cmap='gray')
plt.show()
image3 = cv2.medianBlur(image3, 5)  # 5 is an odd number
image4 = cv2.medianBlur(image4, 5)  # 5 is an odd number
result2_path = 'output/result2.png'
disp_two = get_disp(image3, image4, result2_path)
depth_two= get_depth(350, 15000, 10.3,10.3, disp_two)

In [ ]:
plt.imshow(depth_one, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()


plt.imshow(depth_two, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()
plt.imshow(depth_two_true, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()


plt.imshow(depth_three, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()
plt.imshow(depth_three_true, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()


plt.imshow(depth_four, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()
plt.imshow(depth_four_true, cmap='viridis', interpolation='none')
plt.colorbar(label='Depth (millimetre)')
plt.axis('off')
plt.show()